In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from secret import API_KEY

In [2]:
nbrGames = 20
ig = 'Vayntox'

In [3]:
puuid = requests.get(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{ig}?api_key={API_KEY}").json()["puuid"]

In [4]:
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nbrGames}&api_key={API_KEY}").json()

In [5]:
columns = ['championName', 'win', 'gameDuration', 'teamPosition', 'totalMinionsKilled', 'neutralMinionsKilled', 'totalDamageDealtToChampions', 'goldEarned', 'totalDamageTaken', 'visionScore', 'timeCCingOthers']

In [6]:
array = []

In [7]:
for i in games:
    json = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{i}?api_key={API_KEY}").json()
    index = json['metadata']['participants'].index(puuid)
    player = json['info']['participants'][index]
    player['gameDuration'] = json['info']['gameDuration']
    array.append(list(map(lambda x: player[x], columns)))

In [8]:
df = pd.DataFrame(data=array, columns=columns)

In [9]:
def formatDf(df):
    df['scoreCreeps'] = df['totalMinionsKilled'] + df['neutralMinionsKilled']
    df.rename(columns={'win': 'winrate'})
    return df

In [10]:
formatedDf = formatDf(df)

In [11]:
def filterGames(df, champ):
    return df[df['championName'] == champ]

In [12]:
def getParams(champ):
    def gen(n):
        for i in n:
            yield i

    generator = gen(open('./params.txt', 'r'))
    
    while True:
        str = next(generator)
        if champ in str:
            stats = str.strip().split(';')
            stats.pop(0)
            return {s.split(',')[0]: list(map(float, s.split(',')[1:])) for s in stats}

In [13]:
def score(params, data):
    s = 0
    for k, v in params.items():
        n = (data[k] / (data['gameDuration'] / 60)).mean()
        s += v[0]*((n-v[1])/(v[2]-v[1]))
    return float(s)

In [14]:
def formatScore(nbr):
    level = ""
    if nbr < 0.2:
        level = "C'est pas vraiment pas ouf.."
    elif nbr < 0.4:
        level = "Ça vaaaaaaa"
    elif nbr < 0.6:
        level = "Tu t'en sors bien !"
    elif nbr < 0.8:
        level = "Mec, t'es trop fort"
    else:
        level = "T'es gifted"
    return f"T'es a {nbr:.1%} de performance. {level}"

In [15]:
def getScore(df, champ):
    df = formatDf(df)
    data = filterGames(df, champ)
    params = getParams(champ)
    s = score(params, data)
    return s

In [16]:
res = getScore(formatedDf, 'Vayne')

In [17]:
formatScore(res)

"T'es a 47.4% de performance. Tu t'en sors bien !"

In [18]:
def pieWinrate(df):
    plt.figure()
    plt.title("Winrate")
    percent = df['win'].mean()*100
    labels = ['win', 'lose']
    plt.pie([percent, 100-percent], labels=labels, autopct='%1.0f%%')
    plt.legend()
    plt.show()